In [1]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [142]:
def calibrate_camera(imgs_path, nx, ny):
    images = glob.glob(imgs_path + "/*.jpg")
    # images = os.listdir(imgs_path)
    objpoints, imgpoints = [], []
    objp = np.zeros((nx*ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        if ret is True:
            objpoints.append(objp)
            imgpoints.append(corners)

    return objpoints, imgpoints

In [143]:
def show_image(img):
    cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [144]:
objpoints, imgpoints = calibrate_camera("camera_cal/", 9, 6)

In [145]:
def get_undist_img(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [146]:
def perspective_transform(undist_img):
    offset = 10
    src = np.float32([[566, 457],
                  [750, 457],
                  [1220, undist_img.shape[0]],
                  [130, undist_img.shape[0]]
                 ])

    dst = np.float32([[offset, offset],
                 [undist_img.shape[1] - offset, offset],
                 [undist_img.shape[1] - offset, undist_img.shape[0] - offset],
                 [offset, undist_img.shape[0] - offset]])

    Minv = cv2.getPerspectiveTransform(dst, src)
    M = cv2.getPerspectiveTransform(src, dst)

    warped = cv2.warpPerspective(undist_img, M, undist_img.shape[1::-1], flags=cv2.INTER_LINEAR)
    return warped, M, Minv

In [180]:
img = cv2.imread("test_images_new/1.jpg")

In [181]:
show_image(img)

In [182]:
undist = get_undist_img(img, objpoints, imgpoints)

In [183]:
warped, M, Minv = perspective_transform(undist)

In [241]:
show_image(warped)

In [184]:
def abs_sobel_thresh(img, sobel_kernel=3, orient='x',  thresh=(100,255)):
    if orient=='x':
        sobel = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    if orient=='y':
        sobel = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    sobelabs = np.abs(sobel)
    scaledsobel = np.uint8(255*sobelabs/np.max(sobelabs))
    sbinary = np.zeros_like(scaledsobel)
    sbinary[(scaledsobel >= thresh[0]) & (scaledsobel <= thresh[1])] = 255
    return sbinary

In [185]:
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    magnitude = np.sqrt((sobelx)**2 + (sobely)**2)
    scaledsobel = np.uint8(255*magnitude/np.max(magnitude))
    sbinary = np.zeros_like(scaledsobel)
    sbinary[(scaledsobel >= mag_thresh[0]) & (scaledsobel <= mag_thresh[1])] = 255
    return sbinary

In [186]:
def dir_threshold(img, sobel_kernel=3, thresh=(0.7, 1.3)):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.abs(sobelx)
    abs_sobely = np.abs(sobely)
    direction = np.arctan2(abs_sobely, abs_sobelx)
    binary_output = np.zeros_like(direction)
    binary_output[(direction >= thresh[0]) & (direction <= thresh[1])] = 255
    return binary_output

In [187]:
ksize = 3

In [215]:
# Edit this function to create your own pipeline.
def color_pipeline(img, l_thresh = (150, 255), s_thresh=(150, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2]
    l_channel = hls[:,:,1]
    h_channel = hls[:,:,0]
    
    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Sobel x with l_channel for shadow in the road
    sobelx_l = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0)
    abs_sobelx_l = np.absolute(sobelx_l)
    scaled_sobel_l = np.uint8(255*abs_sobelx_l/np.max(abs_sobelx_l))
    
    # sobelx in s channel
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255

    # sobelx in l channel
    sxbinary_l = np.zeros_like(scaled_sobel_l)
    sxbinary_l[(scaled_sobel_l >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255
    
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 255

    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 255
    
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) 


    combined_binary = np.zeros_like(sxbinary)
    combined_binary[ ((sxbinary == 255) & (sxbinary_l == 255)) | ((s_binary == 255) & (l_binary == 255)) ] = 255

    return combined_binary

In [216]:
combined_binary  = color_pipeline(warped)

In [217]:
show_image(combined_binary)

In [191]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint]) # take the peak value in the left
    #print("Leftx base value: ", leftx_base)
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint  # take the peak value in the right
    #print("Rightx base value: ", rightx_base)
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    #print("Nonzero matrix", nonzero)
    nonzeroy = np.array(nonzero[0]) # y value
    nonzerox = np.array(nonzero[1]) # x value
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
    #    print("win_y_low value : ", win_y_low)
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0, 255, 0), 3)
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0, 255, 0), 3)

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

In [192]:
leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)

In [193]:
show_image(out_img)

In [194]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    return left_fitx, right_fitx, ploty

In [195]:
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

In [209]:
def search_around_poly(undist, binary_warped, Minv, left_fit, right_fit):


    margin = 100


    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) &
                      (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) &
                      (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))


    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]


    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)


    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    lanes_img = np.zeros_like(out_img)

    
    
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])

    pts = np.hstack((pts_left, pts_right))
    
    cv2.fillPoly(lanes_img, np.int_([pts]), (0, 55, 0))
    
    
    # Color in left and right line pixels
    lanes_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [0, 0, 255]
    lanes_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [255, 0, 0]
    
    newwarp = cv2.warpPerspective(lanes_img, Minv, (lanes_img.shape[1], lanes_img.shape[0]))
    
    
    output = cv2.addWeighted(undist, 0.8, newwarp, 1, 0)
    
    
    

    return output

In [218]:
output = search_around_poly(undist, combined_binary, Minv, left_fit, right_fit)

In [219]:
show_image(output)

In [230]:
def proc_pipeline_tmp(img, objpoints, imgpoints):

    undist = get_undist_img(img, objpoints, imgpoints)
    warped_img, M, Minv = perspective_transform(undist)
    combined_binary = color_pipeline(warped_img)
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)
    
    return out_img

In [231]:
def proc_pipeline_video(img, objpoints, imgpoints):

    undist = get_undist_img(img, objpoints, imgpoints)
    warped_img, M, Minv = perspective_transform(undist)
    combined_binary = color_pipeline(warped_img)
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(combined_binary)

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    output = search_around_poly(undist, combined_binary, Minv, left_fit, right_fit)
    
    
    return output

In [232]:
!ls test_videos

challenge.mp4	     harder_challenge_video.mp4  solidWhiteRight.mp4
challenge_video.mp4  project_video.mp4


https://answers.opencv.org/question/200077/combine-several-videos-in-the-same-window-python/

In [237]:
cap1 = cv2.VideoCapture("test_videos/project_video.mp4")


while (cap1.isOpened()):
    ret, frame = cap1.read()
    if ret:
        combined = proc_pipeline_tmp(frame, objpoints, imgpoints)
        full = proc_pipeline_video(frame, objpoints, imgpoints)
        pro = proc_pipeline_tmp(frame, objpoints, imgpoints) 
        pro2 = proc_pipeline_tmp(frame, objpoints, imgpoints)
        
        both = np.concatenate((combined, full), axis=1)
        both2 = np.concatenate((pro, pro2), axis=1)
        all_4 = np.concatenate((both, both2), axis=0)
        cv2.imshow("frame", all_4)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
# Release everything if job is finished
cap1.release()
cap2.release()
cv2.destroyAllWindows()

http://opencv-users.1802565.n2.nabble.com/Combining-Multiple-videos-td2188079.html

In [213]:
def process_video(video_path):
    video_capture = cv2.VideoCapture(video_path)
    while (video_capture.isOpened()):
        ret, frame = video_capture.read()
        if ret:
            output = proc_pipeline_video(frame, objpoints, imgpoints)
            cv2.imshow('frame',output)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    # Release everything if job is finished
    video_capture.release()
    cv2.destroyAllWindows()

In [214]:
process_video("test_videos/challenge.mp4")

In [230]:
!ls test_videos/

challenge.mp4	     harder_challenge_video.mp4  solidWhiteRight.mp4
challenge_video.mp4  project_video.mp4


In [225]:
!ls

camera_cal	 Project_v80519.ipynb  test_images_new
Projectv2.ipynb  Python		       test_videos
Projectv3.ipynb  test_images	       test_videos_output


In [266]:
white_output = 'test_videos_output/project_video_output.mp4'

clip1 = VideoFileClip("test_videos/project_video.mp4")
white_clip = clip1.fl_image(lambda img: proc_pipeline_video(img, objpoints, imgpoints)) 
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_videos_output/project_video_output.mp4.
Moviepy - Writing video test_videos_output/project_video_output.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/project_video_output.mp4
CPU times: user 19min 16s, sys: 3.51 s, total: 19min 20s
Wall time: 16min 13s
